In [2]:
from werkzeug.utils import secure_filename

secure_filename("My Résumé (Final) v2.pdf")

'My_Resume_Final_v2.pdf'

In [ ]:
from app.rag import Return_Poller_Result, embeddings_deployment, openai_client, search_client
import base64

file_url = 

In [5]:
result = Return_Poller_Result(file_url)
markdown_text = result.content

In [42]:
len(result.pages)

6

In [11]:
result.as_dict().keys()

dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'contentFormat', 'sections'])

In [27]:
search_client.get_document_count()

0

In [18]:
from chonkie import RecursiveChunker

chunker = RecursiveChunker(
    chunk_size=512,
    min_characters_per_chunk=100,
)
chunks = chunker.chunk(markdown_text)

In [24]:
response = openai_client.embeddings.create(
            input=[chunk.text for chunk in chunks],
            model=embeddings_deployment
        )

In [33]:
len(response.data)

22

In [28]:
filename = "Django_skills_section.pdf"

In [ ]:
search_documents = []
for i, chunk in enumerate(chunks):
    

In [36]:
search_documents = [{
    "id": base64.urlsafe_b64encode(f"{filename}_{i + 1}".encode()).decode(),
    "content": chunk.text,
    "text_vector": response.data[i].embedding,
    "chunk_index": i + 1,
    "source_url": file_url,
    "source_document": filename
} for i, chunk in enumerate(chunks)]

In [ ]:
# E. Upload to Azure AI Search
if search_documents:
    search_client.upload_documents(documents=search_documents)
    print(f"Uploaded {len(search_documents)} chunks to Azure AI Search.")

Uploaded 22 chunks to Azure AI Search.


In [ ]:
    # Generate embedding for the query
    embedding_response = openai_client.embeddings.create(
        input=query,
        model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
    )
    query_vector = embedding_response.data[0].embedding

    # Create Filter string (OData format)
    filter_str = None
    if filename_filter:
        filter_str = f"source eq '{filename_filter}'"

    # Search Azure AI Search
    results = search_client.search(
        search_text=query,
        vector_queries=[VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="vector")],
        filter=filter_str,
        select=["content", "source"]
    )

    # Format results for the LLM
    context = []
    sources = set()
    for res in results:
        context.append(res['content'])
        sources.add(res['source'])
        
    return json.dumps({"context": "\n\n".join(context), "sources": list(sources)})

In [43]:
from azure.search.documents.models import VectorizableTextQuery

In [50]:
filename

'Django_skills_section.pdf'

In [66]:
query_text = "hihi"
count = 5
vector_query = VectorizableTextQuery(text=query_text, fields="text_vector")
search_query = search_client.search(
        search_text=query_text,  # Keyword search
        vector_queries=[vector_query],  # Vector search
        top=count,  # Limit results
        #filter=f"source_document eq '{filename}'",  # Filtering
        query_type="simple",
        search_fields=["content"],  # Fields to search
        select=['content', 'source_url', 'source_document', 'chunk_index']
    )

query_documents = [{k: v for k, v in doc.items() if not k.startswith("@")} for doc in search_query]

In [67]:
query_documents

[{'source_url': 'https://buddiz.blob.core.windows.net/test-rag-container/lesson.pdf',
  'source_document': 'lesson.pdf',
  'chunk_index': 15,
  'content': 'capability managers love.\n\n\n# Part 4: The Final Presentation (Synthesis)\n\nHere is the answer key for the final recommendation based on the provided dataset logic.\n\n\n## Analysis of the Data:\n\n1\\. Best Sector: If you look at the Heatmap, AI has the darkest squares (highest growth) across most\nregions, but FinTech usually has the best balance of Revenue stability. However, strictly for\n"Risk/Growth Balance," SaaS often shows up as moderate risk (3-5) with consistent growth. Let\'s\n'},
 {'source_url': 'https://buddiz.blob.core.windows.net/test-rag-container/Django_skills_section.pdf',
  'source_document': 'Django_skills_section.pdf',
  'chunk_index': 1,
  'content': 'This is a great teaching scenario. It perfectly illustrates the difference between Data Entry Ease (Method 1) vs. Data Integrity & Normalization (Method 2).\n

In [58]:
query_documents[0]

{'source_url': 'https://buddiz.blob.core.windows.net/test-rag-container/lesson.pdf',
 'source_document': 'lesson.pdf',
 'chunk_index': 15,
 'content': 'capability managers love.\n\n\n# Part 4: The Final Presentation (Synthesis)\n\nHere is the answer key for the final recommendation based on the provided dataset logic.\n\n\n## Analysis of the Data:\n\n1\\. Best Sector: If you look at the Heatmap, AI has the darkest squares (highest growth) across most\nregions, but FinTech usually has the best balance of Revenue stability. However, strictly for\n"Risk/Growth Balance," SaaS often shows up as moderate risk (3-5) with consistent growth. Let\'s\n',
 '@search.score': 0.01666666753590107,
 '@search.reranker_score': None,
 '@search.highlights': None,
 '@search.captions': None}

In [ ]:
sources = {}
for document in query_documents:
    if document['source_document'] in sources:
        temp_list = {"context": []}
        sources[document['source_document']]['context'].append(document['content'])
    else:
        sources[document['source_document']] = {"url": document['source_url'], "context": [document['content']]}

{'lesson.pdf': {'context': ['capability managers love.\n\n\n# Part 4: The Final Presentation (Synthesis)\n\nHere is the answer key for the final recommendation based on the provided dataset logic.\n\n\n## Analysis of the Data:\n\n1\\. Best Sector: If you look at the Heatmap, AI has the darkest squares (highest growth) across most\nregions, but FinTech usually has the best balance of Revenue stability. However, strictly for\n"Risk/Growth Balance," SaaS often shows up as moderate risk (3-5) with consistent growth. Let\'s\n',
   'xytext=(nebula [\'Risk_Score\' ]-1. 5, nebula [ \'Valuation_B\' ] +2) ,\narrowprops=dict (facecolor=\'black\', shrink=0.05)\n)\n\n\\# Reference Line\nax1. axhline (y=10, color=\'grey\', linestyle=\' -- \', alpha=0.5)\nax1. text (x=1, y=10.5, s="Unicorn Threshold (10B) ", color=\'grey\')\n\n· Common Mistake: Forgetting sizes= (min, max) . Without this, the bubbles might be too\nsmall to see differences.\n\n· Detour: Ask the student to change the hue to Region to s

In [ ]:
# --- 3. The Tool Function (The actual RAG retrieval) ---
def search_knowledge_base(query: str, filename: str = None, count: int = 5):
    """
    Retreives relevant context chunks from the Azure AI Search index.
    """
    print(f"🛠️ Tool Triggered: Searching for '{query}' in '{filename}'")
    vector_query = VectorizableTextQuery(text=query, fields="text_vector")
    search_query = search_client.search(
            search_text=query,  # Keyword search
            vector_queries=[vector_query],  # Vector search
            top=count,  # Limit results
            filter=f"source_document eq '{filename}'",  # Filtering
            query_type="simple",
            search_fields=["content"],  # Fields to search
            select=['content', 'source_url', 'source_document', 'chunk_index']
        )

    query_documents = [{k: v for k, v in doc.items() if not k.startswith("@")} for doc in search_query]

    sources = {}
    for document in query_documents:
        if document['source_document'] in sources:
            temp_list = {"context": []}
            sources[document['source_document']]['context'].append(document['content'])
        else:
            sources[document['source_document']] = {"url": document['source_url'], "context": [document['content']]}

    return sources


In [ ]:
@app.post("/ask", response_model=AskResponse)
async def ask_question(request: AskRequest):
    try:
        # Pass the request data to the agent logic
        answer, sources = run_agent(
            user_query=request.query, 
            session_id=request.session_id,
            target_file=request.target_file
        )
        
        return AskResponse(
            answer=answer,
            source=sources
        )
    except Exception as e:
        # Log error for debugging
        print(f"Error in /ask: {e}")
        raise HTTPException(status_code=500, detail=str(e))
    

In [ ]:
if selected_doc:
    st.sidebar.success(f"Active: {selected_doc}")
    # Pass 'selected_doc' to your chat API payload
    # payload = { "query": prompt, "filter_filename": selected_doc }

# 1. Upload New File
uploaded_file = st.sidebar.file_uploader("Upload a PDF", type="pdf")
if uploaded_file:
    if st.sidebar.button("Process & Index"):
        with st.spinner("Uploading and Indexing..."):
            files = {"file": (uploaded_file.name, uploaded_file, "application/pdf")}
            response = requests.post(f"{API_URL}/upload", files=files)
            if response.status_code == 200:
                st.sidebar.success("File Indexed!")
            else:
                st.sidebar.error("Upload failed")

# 2. View Available Files
st.sidebar.subheader("Current Documents")
try:
    files_res = requests.get(f"{API_URL}/files")
    files_list = files_res.json().get("files", [])
    st.sidebar.write(files_list)
except:
    st.sidebar.error("Backend not connected")

# --- Main Chat Interface ---

# Initialize chat history in Streamlit session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# User Input
if prompt := st.chat_input("Ask a question..."):
    # 1. Display user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # 2. Send to FastAPI
    # Prepare Payload
    payload = {
        "query": prompt,
        "session_id": "streamlit_user_1", # Simple ID for now
        "target_file": selected_doc if selected_doc else None
    }

    # Call FastAPI
    with st.chat_message("assistant"):
        try:
            res = requests.post(f"{API_URL}/ask", json=payload)
            data = res.json()
            
            answer = data["answer"]
            sources = data.get("source", [])
            
            st.markdown(answer)
            if sources:
                st.caption(f"📚 Sources: {', '.join(sources)}")
                
            st.session_state.messages.append({"role": "assistant", "content": answer})
            
        except Exception as e:
            st.error(f"Error: {e}")